In [1]:
# basic imports
import pandas as pd
import sys
import numpy as np
import pandas as pd
import scanpy as sc

# add `Tangram` to path
import tangram as tg
import os

In [2]:
## setting for STARmap
sc_file_path = "/home/comp/cszrwang/data/MOB/mob_sc_celltype.h5ad"
spatial_file_path = "/home/comp/cszrwang/data/MOB/mob_st.h5ad"
celltype_key = 'celltype'
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/MOB/Tangram/"
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)

In [3]:
ad_sc = sc.read_h5ad(sc_file_path)
ad_sp = sc.read_h5ad(spatial_file_path)

# use raw count both of scrna and spatial
sc.pp.normalize_total(ad_sc)
celltype_counts = ad_sc.obs[celltype_key].value_counts()
celltype_drop = celltype_counts.index[celltype_counts < 2]
print(f'Drop celltype {list(celltype_drop)} contain less 2 sample')
ad_sc = ad_sc[~ad_sc.obs[celltype_key].isin(celltype_drop),].copy()
sc.tl.rank_genes_groups(ad_sc, groupby=celltype_key, use_raw=False)
markers_df = pd.DataFrame(ad_sc.uns["rank_genes_groups"]["names"]).iloc[0:200, :]
print(markers_df)
genes_sc = np.unique(markers_df.melt().value.values)
print(genes_sc)
genes_st = ad_sp.var_names.values
genes = list(set(genes_sc).intersection(set(genes_st)))

Drop celltype [] contain less 2 sample
     EPL-IN        GC     M/TC     OSNs     PGC
0       App    Grin2b    Olfm1    Gng13   Nrsn1
1      Psap      Meg3  Slc17a7    Rps29    Nppa
2      Rtn1  Il1rapl1    Cdhr1    Rpl41  Nap1l5
3    Atp1b1     Epha5     Reln   Tmsb4x   Nxph1
4      Thy1     Gria2     Chgb   Rpl37a   Grem1
..      ...       ...      ...      ...     ...
195    Cnr1     Man1a   Ogfrl1    Tceb1    Sox2
196    Emc4      Tcf4    Ptprs   Cystm1   Thoc7
197   Tm2d3  Ppp1r12b     Ctsl      Ptn   Uqcrb
198   Lsamp    Stxbp6   Sema3d  Dynlrb2    Sdhb
199   Lrp11  Arhgap20     Clmp  Tmem258  Higd2a

[200 rows x 5 columns]
['A230065H16Rik' 'A830018L16Rik' 'AI593442' 'Abat' 'Abhd12' 'Abhd3'
 'Acbd7' 'Ache' 'Acp1' 'Actg1' 'Adam11' 'Adamts19' 'Adcyap1' 'Adora1'
 'Adrbk2' 'Ahi1' 'Aig1' 'Ajap1' 'Aldoa' 'Aldoc' 'Ampd2' 'Anapc11'
 'Anapc13' 'Ank2' 'Ano3' 'Aplp1' 'Aplp2' 'Apoe' 'Apold1' 'App' 'Arap2'
 'Arglu1' 'Arhgap20' 'Arhgdig' 'Arl3' 'Arl6ip1' 'Arpp21' 'Arxes1' 'Arxes2'
 'Asic1' 'A

In [4]:
tg.pp_adatas(ad_sc, ad_sp, genes=genes)

ad_map = tg.map_cells_to_space(
                   ad_sc,
                   ad_sp,
                   mode='clusters',
                   cluster_label=celltype_key)

# ad_map = tg.map_cells_to_space(
#                    ad_sc,
#                    ad_sp,
#                    mode="constrained",
#                    target_count=ad_sp.obs.cell_count.sum(),
#                    density_prior=np.array(ad_sp.obs.cell_count) / ad_sp.obs.cell_count.sum(),
#                    num_epochs=1000,
#                    cluster_label=celltype_key)

tg.project_cell_annotations(ad_map, ad_sp, annotation=celltype_key)

celltype_density = ad_sp.obsm['tangram_ct_pred']
celltype_density = (celltype_density.T/celltype_density.sum(axis=1)).T

celltype_density.to_csv(output_file_path + '/Tangram_result.csv')

INFO:root:886 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:13895 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 886 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.505, KL reg: 0.328
Score: 0.747, KL reg: 0.000
Score: 0.748, KL reg: 0.000
Score: 0.748, KL reg: 0.000
Score: 0.748, KL reg: 0.000
Score: 0.748, KL reg: 0.000
Score: 0.749, KL reg: 0.000
Score: 0.749, KL reg: 0.000
Score: 0.749, KL reg: 0.000
Score: 0.749, KL reg: 0.000


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
